In [ ]:
#1 - It will trade a small increase in bias for a large decrease in variance 
#2 - F
#3 - F
#4 - The errors are high due to a high bias. You should decrease the regularization parameter 

In [30]:
import boto3 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import accuracy_score 

#defining bucket
s3 = boto3.resource('s3')
bucket_name = 'data445-ashlyn-bucket' 
bucket = s3.Bucket(bucket_name)

#defining csv file
file_key = 'CarPrice_Assignment.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body') 

#reading csv file
car_price = pd.read_csv(file_content_stream)
car_price.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [31]:
#defining input and target variables
X = car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg']]
Y = car_price['price']

#defining list to store
coef = list()

for i in range(0, 1000):

    #splitting data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
    
    #lasso cross validation
    lasso_cv = LassoCV(normalize = True, cv = 5).fit(X_train, Y_train)
    
    #building lasso regression
    lasso_md = Lasso(alpha = lasso_cv.alpha_, normalize = True).fit(X_train, Y_train)
    
    #storing est coef
    coef.append(lasso_md.coef_)
    
#list as data frame
coef_data = pd.DataFrame(coef)

In [32]:
coef_data.head()

,0,1,2,3,4,5,6
0,185.810142,110.808507,297.591348,44.218154,2.097320,-134.800440,-0.0
1,162.186900,120.476041,254.336118,47.422106,2.019428,-48.797876,-0.0
2,162.335041,106.691890,258.639969,57.424335,1.941106,-76.254244,-0.0
3,195.366093,108.940433,308.604990,41.180495,2.068207,-167.796769,-0.0
4,219.291000,94.528741,275.807080,61.277456,1.979409,-66.533295,-0.0


In [21]:
#column 0
sum(coef_data[0] == 0.0)

0

In [22]:
#column 1
sum(coef_data[1] == 0.0)

0

In [23]:
#column 2
sum(coef_data[2] == 0.0)

0

In [24]:
#column 3
sum(coef_data[3] == 0.0)

0

In [25]:
#column 4
sum(coef_data[4] == 0.0)

0

In [26]:
#column 5
sum(coef_data[5] == 0.0)

15

In [27]:
#column 6
sum(coef_data[6] == 0.0)

756

In [33]:
#dropping highway
X = car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg']]
Y = car_price['price']

#splitting data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [34]:
def l2_normalization(X):
   
    x_mean = np.mean(X)
    l2 = np.sqrt(sum(X**2))
    return (X - x_mean) / l2

In [35]:
#building linear regression model
lm_md = LinearRegression().fit(X_train, Y_train)

#predicting on the test dataset
md1_pred = lm_md.predict(X_test)

#computing mse
mse1 = np.mean(np.power(Y_test - md1_pred, 2))
mse1

12068736.70322945

In [36]:
#ridge regression model
ridge_cv = RidgeCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100], cv = 5).fit(X_train, Y_train)
ridge_alpha = ridge_cv.alpha_
ridge_alpha

100.0

In [40]:
#defining list to store
coef2 = list()

for i in range(0, 100):

    #splitting data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
    
    ridge_cv = RidgeCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100], cv = 5).fit(X_train, Y_train)
    
    #building ridge regression
    ridge_md = Ridge(alpha = ridge_cv.alpha_, normalize = True).fit(X_train, Y_train)
    
    #storing est coef
    coef2.append(ridge_md.coef_)
    
#list as data frame
coef2_data = pd.DataFrame(coef2)

In [41]:
coef2_data.head()

,0,1,2,3,4,5
0,60.288380,13.280803,6.850145,12.817608,-0.092275,-71.235324
1,116.274548,112.131815,323.954766,60.570272,2.207717,-97.563068
2,7.195291,1.676090,2.487752,1.677963,-0.008076,-8.609071
3,7.819565,1.577582,1.407503,1.504966,-0.008872,-7.943423
4,7.730404,1.616010,0.349455,1.604976,-0.012589,-8.207905


In [42]:
#building the ridge model
ridge_md = Ridge(alpha = ridge_alpha).fit(X_train, Y_train)

#predicting on test dataset
md2_pred = ridge_md.predict(X_test)
md2_pred

#computing the mse
mse2 = np.mean(np.power(md2_pred - Y_test, 2))
mse2

5283130.917752258

In [ ]:
#Based on the 2 MSEs, I would choose the linear regression
#model to predict car prices bc it is smaller than the 
#ridge model mse